In [1]:
import cv2
import os
import shutil
import matplotlib.pyplot as plt
from keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Updated import
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D
from keras.models import Model
from tensorflow.keras.applications import DenseNet121

In [2]:
import os
from sklearn.model_selection import train_test_split

In [3]:
image_dir = r"C:\Users\laptop\AI Huawei\26_8_2024\Apple DS"  # Replace with your actual image directory
class_names = ["Apple_BR", "Apple_CR", "Apple_HE"]  # Replace with your actual class names

In [4]:
import os
import numpy as np
from PIL import Image

# Initialize empty lists for storing image paths and labels
image_paths = []
labels = []

# Loop through each class directory
for class_name in class_names:
    class_dir = os.path.join(image_dir, class_name)
    for image_path in os.listdir(class_dir):
        image_paths.append(os.path.join(class_dir, image_path))
        labels.append(class_name)       

In [5]:
X_train, X_test, y_train, y_test = train_test_split(image_paths, labels, test_size=0.2, random_state=42, stratify=labels)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

In [6]:
print("Length of X_train:", len(X_train))
print("Length of X_test:", len(X_test))
print("Length of X_val:", len(X_val))

Length of X_train: 172
Length of X_test: 54
Length of X_val: 44


In [7]:
import os

train_dir = r"C:\Users\laptop\AI Huawei\26_8_2024\Splitting DS\Train"  # Replace with your desired path
test_dir = r"C:\Users\laptop\AI Huawei\26_8_2024\Splitting DS\Test"  # Replace with your desired path
val_dir =  r"C:\Users\laptop\AI Huawei\26_8_2024\Splitting DS\Valid"  # Replace with your desired path

os.makedirs(train_dir, exist_ok=True)  # Create directories if they don't exist
os.makedirs(test_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

In [8]:
for image_path, label in zip(X_val, y_val):
    class_dir = os.path.join(val_dir, label)  # Construct class-specific subdirectory
    os.makedirs(class_dir, exist_ok=True)  # Create class directory if it doesn't exist
    shutil.copy(image_path, class_dir)  # Copy image to the class directory

In [9]:
for image_path, label in zip(X_train, y_train):
    class_dir = os.path.join(train_dir, label)  # Construct class-specific subdirectory
    os.makedirs(class_dir, exist_ok=True)  # Create class directory if it doesn't exist
    shutil.copy(image_path, class_dir)  # Copy image to the class directory

In [10]:
for image_path, label in zip(X_test, y_test):
    class_dir = os.path.join(test_dir, label)  # Construct class-specific subdirectory
    os.makedirs(class_dir, exist_ok=True)  # Create class directory if it doesn't exist
    shutil.copy(image_path, class_dir)  # Copy image to the class directory

In [11]:
# dimensions of our images.
img_width, img_height = 224, 224
batch_size = 5  

train_datagen = ImageDataGenerator(
        rescale=1./255,            # normalize pixel values to [0,1]
        shear_range=0.2,      
        zoom_range=0.2,    
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True)  


val_datagen = ImageDataGenerator(
         rescale=1./255)       # normalize pixel values to [0,1]

import numpy
numpy.random.seed(42)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 172 images belonging to 3 classes.
Found 44 images belonging to 3 classes.


In [12]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

# Define the input shape
input_shape = (224, 224, 3)

# Load the DenseNet121 model pretrained on ImageNet, specifying the input shape
base_model = DenseNet121(weights='imagenet', input_shape=input_shape)


# Build the model
model = Sequential([
    Input(shape=input_shape),
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=50,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=5,
    callbacks=[lr_scheduler],
    class_weight={0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0}
)


29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 12s 0us/step
Epoch 1/20


C:\Users\laptop\AppData\Roaming\Python\Python39\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


35/50 ━━━━━━━━━━━━━━━━━━━━ 6s 415ms/step - accuracy: 0.2800 - loss: 11.6792

C:\ProgramData\Anaconda3\lib\contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


50/50 ━━━━━━━━━━━━━━━━━━━━ 51s 387ms/step - accuracy: 0.3041 - loss: 11.6254 - val_accuracy: 0.4000 - val_loss: 11.2172 - learning_rate: 1.0000e-05
Epoch 2/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 16s 302ms/step - accuracy: 0.4223 - loss: 11.2012 - val_accuracy: 0.6316 - val_loss: 10.8703 - learning_rate: 1.0000e-05
Epoch 3/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 16s 306ms/step - accuracy: 0.5352 - loss: 10.9908 - val_accuracy: 0.8800 - val_loss: 10.5746 - learning_rate: 1.0000e-05
Epoch 4/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 16s 301ms/step - accuracy: 0.7149 - loss: 10.6217 - val_accuracy: 0.9474 - val_loss: 10.3267 - learning_rate: 1.0000e-05
Epoch 5/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 16s 306ms/step - accuracy: 0.6817 - loss: 10.5208 - val_accuracy: 0.9600 - val_loss: 10.1238 - learning_rate: 1.0000e-05
Epoch 6/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 16s 311ms/step - accuracy: 0.8446 - loss: 10.2658 - val_accuracy: 1.0000 - val_loss: 9.9570 - learning_rate: 1.0000e-05
Epoch 7/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 17s 322ms/step - acc

In [13]:
from sklearn.preprocessing import LabelEncoder

# Use LabelEncoder to encode 'no' as 0 and 'yes' as 1
label_encoder = LabelEncoder()
y_test_encoded = label_encoder.fit_transform(y_test)

# Print the encoded labels
print("Encoded Labels:")
print(y_test_encoded)

Encoded Labels:
[1 1 0 1 0 2 2 2 2 2 1 2 1 2 2 2 0 0 1 1 0 0 1 0 1 2 0 1 0 1 2 0 0 2 0 0 2
 2 2 1 1 0 2 1 1 1 1 0 2 1 0 0 2 0]


In [14]:
test_aug = ImageDataGenerator(
    # Rescale
    rescale=1/255.0
)

# Read data from directory
test_data = test_aug.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode="categorical"
)

Found 54 images belonging to 3 classes.


In [15]:
model.evaluate(test_data)


C:\Users\laptop\AppData\Roaming\Python\Python39\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 220ms/step - accuracy: 1.0000 - loss: 8.4520


[8.443034172058105, 1.0]

In [16]:
import numpy as np

# Assuming test_data is a tf.data.Dataset or similar that yields (input, label) pairs
#y_true = np.concatenate([y for x, y in test_data], axis=0)
y_pred_prob = model.predict(test_data)
y_pred = np.argmax(y_pred_prob, axis=1)  # for multi-class classification
y_pred_prob

11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 585ms/step


array([[2.89183240e-02, 3.23723145e-02, 9.38709438e-01],
       [8.91482592e-01, 6.29868126e-03, 1.02218725e-01],
       [9.90735590e-01, 7.26958364e-03, 1.99473742e-03],
       [8.17273092e-03, 1.63966343e-02, 9.75430667e-01],
       [9.89793420e-01, 6.76004635e-03, 3.44661763e-03],
       [5.71826957e-02, 4.59778532e-02, 8.96839380e-01],
       [8.50723624e-01, 2.64149681e-02, 1.22861437e-01],
       [9.61501479e-01, 2.44850107e-02, 1.40134981e-02],
       [9.81241941e-01, 4.41999082e-03, 1.43380798e-02],
       [1.76588036e-02, 9.64388192e-01, 1.79529972e-02],
       [9.09194164e-03, 7.40374625e-02, 9.16870534e-01],
       [5.59415435e-03, 9.93667781e-01, 7.38016504e-04],
       [6.50941730e-01, 2.71261066e-01, 7.77971894e-02],
       [3.37550929e-03, 9.52828825e-01, 4.37955819e-02],
       [2.00529583e-02, 3.38292122e-02, 9.46117818e-01],
       [7.54062116e-01, 7.44972527e-02, 1.71440586e-01],
       [9.74881172e-01, 2.48148181e-02, 3.04056943e-04],
       [1.03528805e-01, 1.56422

In [17]:
import time
import numpy as np

# Assuming test_data is a tf.data.Dataset or similar that yields (input, label) pairs

# Start timing for the prediction process
start_time = time.time()

# Get predicted probabilities from the model
y_pred_prob = model.predict(test_data)

# End timing for the prediction process
end_time = time.time()

# Calculate the time taken
time_taken = end_time - start_time

# Convert probabilities to class predictions for multi-class classification
y_pred = np.argmax(y_pred_prob, axis=1)

# Print the predicted probabilities and the time taken
print(y_pred_prob)
print(f"Time taken for prediction: {time_taken} seconds")


11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 247ms/step
[[5.49598992e-01 2.40021925e-02 4.26398784e-01]
 [5.55036217e-03 2.83861030e-02 9.66063559e-01]
 [1.49297854e-03 9.93976831e-01 4.53013042e-03]
 [7.05419620e-03 9.91240919e-01 1.70487526e-03]
 [2.16694120e-02 4.66479454e-03 9.73665833e-01]
 [1.11540310e-01 2.08388027e-02 8.67620885e-01]
 [8.94176602e-01 9.25285593e-02 1.32948365e-02]
 [8.17273092e-03 1.63966343e-02 9.75430667e-01]
 [5.72904408e-01 5.09302244e-02 3.76165301e-01]
 [6.50941730e-01 2.71261066e-01 7.77971894e-02]
 [2.00529583e-02 3.38292122e-02 9.46117818e-01]
 [9.70042229e-01 2.58594640e-02 4.09827894e-03]
 [7.33001810e-03 5.88023588e-02 9.33867633e-01]
 [9.74881172e-01 2.48148181e-02 3.04056943e-04]
 [8.24424803e-01 1.63889170e-01 1.16860736e-02]
 [9.92024422e-01 4.24428610e-03 3.73124098e-03]
 [5.75207314e-03 2.98612751e-02 9.64386642e-01]
 [1.86674166e-02 3.56440991e-02 9.45688546e-01]
 [1.96823496e-02 9.75675702e-01 4.64191427e-03]
 [1.78814419e-02 2.21894514e-02 9.59929049e-01]

In [18]:
import time
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
import numpy as np
import tensorflow as tf

# Path to your image
img_path = 'BLAST1_001.jpg'

# Load the image with target size matching the input shape of the model
img = image.load_img(img_path, target_size=(224, 224))

# Convert the image to a numpy array
img_array = image.img_to_array(img)

# Start timing for the entire process (preprocessing + prediction)
total_start_time = time.time()


# Start timing for preprocessing
preprocess_start_time = time.time()

# Expand dimensions to match the expected input shape of the model (1, 400, 400, 3)
img_array = np.expand_dims(img_array, axis=0)

# Preprocess the image for MobileNetV2
img_array = preprocess_input(img_array)

# End timing for preprocessing
preprocess_end_time = time.time()
preprocess_time_taken = preprocess_end_time - preprocess_start_time

# Start timing for prediction
start_time = time.time()

# Make a prediction using the MobileNetV2 model
predictions = model.predict(img_array)

# End timing for prediction
end_time = time.time()
prediction_time_taken = end_time - start_time

# End timing for the entire process
total_end_time = time.time()
total_time_taken = total_end_time - total_start_time

# Print the predictions and time taken
print(predictions)
print(f"Time taken for preprocessing: {preprocess_time_taken} seconds")
print(f"Time taken for prediction: {prediction_time_taken} seconds")
print(f"Total time taken (preprocessing + prediction): {total_time_taken} seconds")


FileNotFoundError: [Errno 2] No such file or directory: 'BLAST1_001.jpg'